# Bayesian Optimization on OpenSearch Boosts & Params

This notebook is from Doug Turnbull's [Twitch Livecoding](https://www.twitch.tv/videos/1236075888) where he live-coded bayesian optimization. It's messy code that you'd expect from a live coding session, so caveat emptor :)

Bibliography & Further reading

* [Exploring Bayesian Optimization](https://distill.pub/2020/bayesian-optimization) by Apoorv Agnihotri and Nipun Batra of the Indian Institute of Technology Gandhinagar
* [Improving search relevance with data-driven query optimization](https://www.elastic.co/blog/improving-search-relevance-with-data-driven-query-optimization) by Josh Devins, Senior Principal Engineer, Elastic
* [AI Powered Search](http://aipoweredsearch) by Trey Grainger, Doug Turnbull, and Max Irwin. In particular chapter 12 uses Bayesian Optimization techniquest to overcome presentation bias.

## Setup & Indexing

We are using [TheMovieDB](http://themoviedb.org) corpus. Download and index that to the local Elasticsearch.

In [3]:
from ltr.client import OpenSearchClient
client = OpenSearchClient()

In [4]:
from ltr import download
corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments.txt'

download([corpus, judgments], dest='data/');

data/tmdb.json already exists
data/title_judgments.txt already exists


In [5]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

Index tmdb already exists. Use `force = True` to delete and recreate


```
grade/label, query, doc_id
```

grade = 0-4 with 0 completely irrelevant, 4 absolutely relevant
query = rambo
doc_id = movie id "tmdb id"

```
```

## Read a simple movie judgment list

These judgments have a grade (0-4) that says how relevant a movie is (the 'doc id') for a query (ie 'rambo', etc)

In [6]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open, judgments_to_dataframe
from itertools import groupby

judgments_dataframe = None

with judgments_open('data/title_judgments.txt') as judgment_list:
    print(dir(judgment_list))
    judgments_dataframe = judgments_to_dataframe(judgment_list)
    
judgments_dataframe

Recognizing 40 queries
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'f', 'judgments', 'keywords', 'kw_with_weight']


/Users/markcoh/IdeaProjects/OpenSearchWork/hello-ltr/ltr/judgments.py:266: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



,uid,qid,keywords,docId,grade
0,1_7555,1,rambo,7555,4
1,1_1370,1,rambo,1370,3
2,1_1369,1,rambo,1369,3
3,1_13258,1,rambo,13258,2
4,1_1368,1,rambo,1368,4
...,...,...,...,...,...
1385,40_37079,40,star wars,37079,0
1386,40_126757,40,star wars,126757,0
1387,40_39797,40,star wars,39797,0
1388,40_18112,40,star wars,18112,0


In [7]:
# DCG - discounted cumulative gain
# ERR, MRR, etc
# Cumulative Gain - Precision
#  - sum of the top N grades for a result set


In [8]:
{judgment['docId']: judgment for judgment in judgments_dataframe['rambo' == judgments_dataframe['keywords']].to_dict('records')}

{'7555': {'uid': '1_7555',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '7555',
  'grade': 4},
 '1370': {'uid': '1_1370',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '1370',
  'grade': 3},
 '1369': {'uid': '1_1369',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '1369',
  'grade': 3},
 '13258': {'uid': '1_13258',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '13258',
  'grade': 2},
 '1368': {'uid': '1_1368',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '1368',
  'grade': 4},
 '31362': {'uid': '1_31362',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '31362',
  'grade': 1},
 '61410': {'uid': '1_61410',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '61410',
  'grade': 1},
 '319074': {'uid': '1_319074',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '319074',
  'grade': 0},
 '10296': {'uid': '1_10296',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '10296',
  'grade': 0},
 '35868': {'uid': '1_35868',
  'qid': 1,
  'keywords': 'rambo',
  'docId': '35868',
  'grade': 0},
 '131457': {'uid': 

In [9]:
import json


## Evaluate a query template using average CG@10

We use a parameteried query 'template' (where we do the replacement here) and see what the Cumulative Gain (average relevance grade) for the solution across all queries

In [10]:
def search_and_evaluate(judgments, es_query, at=10, params={}):
    query = json.dumps(es_query)
    
    for param, value in params.items():
        query = query.replace("{{" + param + "}}", str(value))
   
    es_query = json.loads(query)
    
    average_cumulative_gain = 0.0
    
    for keywords in judgments['keywords'].unique():
        query = json.dumps(es_query).replace("{{keywords}}", keywords)
        query = json.loads(query)
        
        # print(json.dumps(query))
     
        results = client.es.search(index='tmdb', body=query)
        # print(keywords)
        # print('-----')
        this_keyword_judgments = {judgment['docId']: judgment 
                                  for judgment in 
                                  judgments[keywords == judgments['keywords']].to_dict('records')}
        cumulative_gain = 0 
        for idx, hit in enumerate(results['hits']['hits']):
            this_grade = 0.0
            try:
                this_grade = this_keyword_judgments[hit['_id']]['grade']
                cumulative_gain += this_grade
            except KeyError:
                pass
            if idx >= at:
                break

            # print(hit['_source']['title'], this_grade)
            
        cumulative_gain /= at   # < now this is the average grade for top at
        average_cumulative_gain += cumulative_gain
        
        # print(f"CG@10 {cumulative_gain}")
        # print()
    return average_cumulative_gain / len(judgments['keywords'].unique())
        
search_and_evaluate(judgments_dataframe, es_query={'query': {'match': {'title': '{{keywords}}'}}})


0.8825000000000001

## Simple query template to optimize

Query template to optimize

Notice the naive grid search (two for loops) would be too slow if we tried every value. Running every scenario against Elasticsearch would be too slow. So we prime our optimization with a handful of values.

In [11]:


es_query={'query':
  {
      "bool": {
          "should": [
              {'match': 
                  {'title': 
                     {
                       'query': '{{keywords}}',
                       'boost': '{{title_boost}}'
                     }

                   }
              },
              {'match': 
                  {'overview': 
                     {
                       'query': '{{keywords}}',
                       'boost': '{{overview_boost}}'
                     }

                   }
              }
          ]
      }
  }
   
}


# Grid search

runs = []
for title_boost in range(0,200,50):  # Random -> Generate random value
    for overview_boost in range(0, 200, 50):
        print("-----------------------------------------")
        params={'title_boost': title_boost, 'overview_boost': overview_boost}
        avg_cg = search_and_evaluate(judgments_dataframe, es_query=es_query, 
                                     params=params)
        print(f"--- RUN {avg_cg} {repr(params)}")
        
        runs.append({**params, **{'mean_cg': avg_cg}})
              
sorted_by_perf = sorted(runs, key=lambda value: value['mean_cg'], reverse=True)
sorted_by_perf
# 40,000 * k queries we're hitting 

-----------------------------------------
--- RUN 0.36500000000000005 {'title_boost': 0, 'overview_boost': 0}
-----------------------------------------
--- RUN 0.4675000000000001 {'title_boost': 0, 'overview_boost': 50}
-----------------------------------------
--- RUN 0.4675000000000001 {'title_boost': 0, 'overview_boost': 100}
-----------------------------------------
--- RUN 0.4675000000000001 {'title_boost': 0, 'overview_boost': 150}
-----------------------------------------
--- RUN 0.9100000000000001 {'title_boost': 50, 'overview_boost': 0}
-----------------------------------------
--- RUN 0.8724999999999999 {'title_boost': 50, 'overview_boost': 50}
-----------------------------------------
--- RUN 0.6775 {'title_boost': 50, 'overview_boost': 100}
-----------------------------------------
--- RUN 0.63 {'title_boost': 50, 'overview_boost': 150}
-----------------------------------------
--- RUN 0.9100000000000001 {'title_boost': 100, 'overview_boost': 0}
----------------------------

[{'title_boost': 100, 'overview_boost': 50, 'mean_cg': 0.9525},
 {'title_boost': 150, 'overview_boost': 50, 'mean_cg': 0.9400000000000001},
 {'title_boost': 150, 'overview_boost': 100, 'mean_cg': 0.9325000000000001},
 {'title_boost': 50, 'overview_boost': 0, 'mean_cg': 0.9100000000000001},
 {'title_boost': 100, 'overview_boost': 0, 'mean_cg': 0.9100000000000001},
 {'title_boost': 150, 'overview_boost': 0, 'mean_cg': 0.9100000000000001},
 {'title_boost': 50, 'overview_boost': 50, 'mean_cg': 0.8724999999999999},
 {'title_boost': 100, 'overview_boost': 100, 'mean_cg': 0.8724999999999999},
 {'title_boost': 150, 'overview_boost': 150, 'mean_cg': 0.8724999999999999},
 {'title_boost': 100, 'overview_boost': 150, 'mean_cg': 0.72},
 {'title_boost': 50, 'overview_boost': 100, 'mean_cg': 0.6775},
 {'title_boost': 50, 'overview_boost': 150, 'mean_cg': 0.63},
 {'title_boost': 0, 'overview_boost': 50, 'mean_cg': 0.4675000000000001},
 {'title_boost': 0, 'overview_boost': 100, 'mean_cg': 0.46750000000

## Train the Gaussian Process on runs so far

We want to learn the best places to explore for more optimal relevance (`mean_cg`). So we train a [Gaussian Process](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html) to learn CG as a function of the parameters (`title_boost`, `overview_boost`).

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
import pandas as pd

runs_so_far = pd.DataFrame(sorted_by_perf)

y_train = runs_so_far['mean_cg']
x_train = runs_so_far[['title_boost', 'overview_boost']]


gpr = GaussianProcessRegressor()
gpr.fit(x_train.to_numpy(), y_train.to_numpy())

## A Gaussian Process gives us BOTH uncertainty and prediction

A GaussianProcess learns BOTH a prediction and the uncertainty of that prediction. As we move farther from the training data, the prediction becomes less certain.

On any given data point we can get the prediction and the uncertainty in that prediction (as standard deviation of the prediction at that point).

First we look at a value we trained on, notice the very low std deviation

In [ ]:
import numpy as np

prediction, std_dev = gpr.predict([[100.0, 50.0]], return_std=True)
prediction, std_dev

Much more uncertainty in values we haven't seen

In [ ]:
prediction, std_dev = gpr.predict([[300.0, 50.0]], return_std=True)
prediction, std_dev

## Bayesian optimization - find wher eto explore

With a series of probe points, we can see the prediction and standard deviations in a Pandas Dataframe below.

In [ ]:
probe_point = np.array([[0.0, 150.0], [45.0, 175.0]])
probe_point

In [ ]:
predictions, std_devs = gpr.predict(probe_point, return_std=True)


together = []
for i in range(len(predictions)):
    together.append({'prediction': predictions[i],
                     'std_dev': std_devs[i]})
    

explore_points = pd.DataFrame(together) 
explore_points

## Probability of improvement Scoring

The goal of Bayesian Optimization is to score the value of one of the probe points for more expensive probing with Elasticsearch. The idea is we have a cheap model that can give us a guess (the Gaussian Process) and an expensive way to get a ground truth (run many queries against Elasticsearch).

We'll score using the probability of improvement. The first componend is the `opportunity` - how much would be gained by the current prediction (regardless of the uncertainty in that prediction).

We add that to the dataframe here.

In [ ]:
best_cg = 0.9525  # Best CG we've seen in the current data points

explore_points['opportunity'] = explore_points['prediction'] - best_cg
explore_points

### Divide by std dev to get the prob of improvement

If the uncertainty is high, then the value approaches 0. If the uncertainty is low, then the value increases quite a bit. 

`norm.cdf` scales this between 0-1 to give us more of a probability.

In [ ]:
from scipy.stats import norm

std_dev = 0.00000001

norm.cdf( (explore_points['opportunity']) / std_dev)

In [ ]:
std_dev = 0.999999

norm.cdf( (explore_points['opportunity']) / std_dev)

## Prob of improvement for each `explore_point`

In [ ]:
norm.cdf( explore_points['opportunity'] / explore_points['std_dev'])

## Score 40k points to find best explore points

In [ ]:

probe_point = np.array([[0.0, 150.0], [45.0, 175.0]])
probes = []
for title_boost in range(0,200,1):  # Random -> Generate random value
    for overview_boost in range(0, 200, 1):
        probes.append([title_boost, overview_boost])
        
probes = pd.DataFrame(probes, columns=['title_boost', 'overview_boost'])

predictions, std_devs = gpr.predict(probes, return_std=True)

predictions, std_devs

In [ ]:
probes

In [ ]:
together = []
for i in range(len(predictions)):
    together.append({'prediction': predictions[i],
                     'std_dev': std_devs[i]})
    

explore_points = pd.DataFrame(together) 
explore_points

### Theta is used as an explore / exploit parameter

Higher theta means we drown out the opportunity, and we bias towards exploring untried points. Lower theta means we try areas close to areas we've already explored.

In [ ]:
best_cg = 0.9525

theta = 20.0
explore_points['opportunity'] = explore_points['prediction'] - best_cg - theta
explore_points

explore_points.sort_values(by='opportunity', ascending=False)

### Observe highest probability of improvement

In [ ]:
explore_points['prob_of_improvement'] = norm.cdf( explore_points['opportunity'] / explore_points['std_dev'])
explore_points.sort_values(by='prob_of_improvement', ascending=False)

In [ ]:
top_places_to_try = explore_points.sort_values(by='prob_of_improvement', ascending=False).head(10)
top_places_to_try

In [ ]:
best_probes = probes.loc[top_places_to_try.index]
best_probes

## Take top N best to Elasticsearch

We can now try the highest scored probe points

In [ ]:
best_probes = best_probes.to_dict(orient='records')

for probe in best_probes:
    print(probe)
    print("-----------------------------------------")
    avg_cg = search_and_evaluate(judgments_dataframe, es_query=es_query, 
                                 params=probe)
    print(f"--- RUN {avg_cg} {repr(params)}")

    runs.append({**probe, **{'mean_cg': avg_cg}})
              
sorted_by_perf = sorted(runs, key=lambda value: value['mean_cg'], reverse=True)
sorted_by_perf

sorted_by_perf 

## Retrain Gaussian Process on every direct observation

We repeat the process by retraining the Gaussian Process on **every observation thusfar**

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
import pandas as pd

runs_so_far = pd.DataFrame(sorted_by_perf)

y_train = runs_so_far['mean_cg']
x_train = runs_so_far[['title_boost', 'overview_boost']]


gpr = GaussianProcessRegressor()
gpr.fit(x_train.to_numpy(), y_train.to_numpy())

## Repeat prob of improvement scoring, reprobe Elasticsearch, etc

Now we repeat the scoring on the Gaussian Process trained on this larger dataset. With more data, we can probe more accurately. Repeat until you feel you get a sense of the optimum :)

## Next steps

* Probability of Improvement is usually not the preferred scoring. It just computes a probability. But what you really want is sto get the **expected improvement**. A score that accounts for cases where the improvement would be dramatically higher. So a low probability of a high increase in our relevance stat - CG - might be scored higher!